# Amazon Bedrock Batch Inference

## 背景
- Bedrockのbatch推論のquotaがon demandと関係ない（だからquotaに引っ掛からなくてうれしい）と言う説
- text to image (SDXL or Titan or Both)で500枚くらいバッチ推論ジョブを発行してどんな風に実行が完了するか（もしくは完了しないか）を見てみる
- On demandのquotaより明らかに早かったら嬉しい


## 検証条件
- 推論方式: バッチ
- 生成: テキスト → 画像
- モデル:
    - amazon.titan-image-generator-v1
    - stability.stable-diffusion-xl-v1

## 結論

### 入出力データ
- インターフェイスはS3のみ。
- バッチ推論は入出力ファイルサイズ制限がある。モデル間での差分はない。
- https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html#quotas-batch
- 最小も最大も定義されている。

#### 入力データフォーマット
- 一枚一枚に対してはオンデマンド推論と同じ。それをJSONLにしてS3に置く。
- リクエスト枚数が少ないと、ジョブ実行時の序盤にエラーで跳ね返される。
    - Titan: 4枚
    - SD: 10枚

#### 出力データフォーマット
それぞれのモデルで出力方法が全く異なるので注意

- Titan：単一のJSONLにバイナリで全画像が入力情報とセットで埋まっている。大量画像入っているのでデカJSONL
- SD：ファイル名に連番が入った一枚一枚のPNG

入力と出力の突き合わせ

- Titan：1行ずつの入力と出力がすぐ対応できるので、突き合わせしやすい
- SD：連番で突き合わせ作業するしかない

### ジョブ
- ジョブは並列実行されない。
- 複数のジョブを投げることは可能だが、順序保証されて（Undocumented）キューイングされて順次処理される。したがって前のジョブが完了するまで次のジョブは実行されない。
- よってジョブの並列実行による高速化はされない。
- またバッチ推論にすることでオンラインより高速化もされない。
- バッチ推論ジョブの速度はオンデマンド推論を同期処理しているような速度のため、非同期でオンデマンド処理させた方が高速化は可能。

### 結果
600枚全て成功

- Titan: 12 sec / image
- SD: 7 sec /image　← 70分くらい

条件全て揃えられているわけではないので注意

## 公開情報
- 開発者ドキュメント: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference.html
- Quota: https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html#quotas-batch
- コードサンプル: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-example.html

## 環境準備

### 2024.2.1
- Public Preview
- 利用方法
    - REST API: あると思うが面倒
    - CLI: 無さそう
    - SDK: プレビューのがある
        - PythonとJavaのみ
        - ただし他サービスと同様、プレビュー状態のAPIは一般公開SDKに含まれていない
        - そのため以下の公式whlからインストールが必要
        - https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip
    - コンソール: 無さそう

Python SDKを使うこととし、以下でSDKのインストールをする

In [ ]:
! chmod +x install_sdk.sh
! ./install_sdk.sh

上記実行後、カーネルの再起動が必要

In [ ]:
from IPython import Application
Application.instance().kernel.do_shutdown(True)

```aws s3 cp``` などをするため、AWS CLIがあるか確認

In [ ]:
! aws --version

## 入力データフォーマット

参考: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-data.html

以下、サンプルの入力JSON

入力JSON Linesフォーマット
```JSON
{
    "recordId": "12 character alphanumeric string",
    "modelInput": {JSON body}
}
...
{
    "recordId": "12 character alphanumeric string",
    "modelInput": {JSON body}
}
```

Titan imageの場合の推論入力JSON: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html#model-parameters-titan-image-api


```JSON
{
    "inputText": string,
    "textGenerationConfig": {
        "temperature": float,  
        "topP": float,
        "maxTokenCount": int,
        "stopSequences": [string]
    }
}
```

Tiatanの場合のバッチ推論入力JSON Lines
```JSON
{
    "recordId": "12 character alphanumeric string",
    "modelInput": {
        "inputText": string,
        "textGenerationConfig": {
            "temperature": float,  
            "topP": float,
            "maxTokenCount": int,
            "stopSequences": [string]
        }
    }
}
...
{
    "recordId": "12 character alphanumeric string",
    "modelInput": {
        "inputText": string,
        "textGenerationConfig": {
            "temperature": float,  
            "topP": float,
            "maxTokenCount": int,
            "stopSequences": [string]
        }
    }
}
```

## 共通条件設定

In [ ]:
image_size:int = 1024
cfg_scale:int = 8.0
seed:int = 0

In [ ]:
height:int = image_size
width:int = image_size

image_generation_config:dict = {
    "numberOfImages": 1,
    "quality": "standard",
    "height": height,
    "width": width,
    "cfgScale": cfg_scale,
    "seed": seed
}

def return_body_for_titan_image(prompt:str) -> dict:
    return {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt
        },
        "imageGenerationConfig": image_generation_config
    }

def return_body_for_sd(prompt:str) -> dict:
    return {
        "text_prompts": [{"text": prompt}],
        "height": height,
        "width": width,
        "cfg_scale": cfg_scale,
        # "clip_guidance_preset": string,
        # "sampler": string,
        # "samples",
        "seed": seed,
        # "steps": int,
        # "style_preset": string,
        # "extras": JSON object
    }

functions = {
    "amazon.titan-image-generator-v1": return_body_for_titan_image,
    "stability.stable-diffusion-xl-v1": return_body_for_sd,
}

## 単発推論
まずは単発を確認

In [ ]:
prompt:str = "A dog running at a park."

from func import generate_image_by

### Titan image
https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html

In [ ]:
body = {
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": prompt
    },
    "imageGenerationConfig": image_generation_config
}
generate_image_by(model_id="amazon.titan-image-generator-v1", body=body)

### Stable Diffusion
https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-diffusion-1-0-text-image.html

In [ ]:
body = {
    "text_prompts": [{"text": prompt}],
    "height": height,
    "width": width,
    "cfg_scale": cfg_scale,
    # "clip_guidance_preset": string,
    # "sampler": string,
    # "samples",
    "seed": seed,
    # "steps": int,
    # "style_preset": string,
    # "extras" :JSON object
}
generate_image_by(model_id="stability.stable-diffusion-xl-v1", body=body)

## 検証

### 条件設定

バッチ推論は入出力ファイルサイズ制限がある。モデル間での差分はない。
https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html#quotas-batch

最小も最大も定義されている。そのためリクエスト枚数が少ないと、ジョブ実行時の序盤にエラーで跳ね返される。

- Titan: 4枚
- SD: 10枚

程度が最小。

### ジョブ作成

In [ ]:
import bedrock
_bedrock:bedrock.Bedrock = bedrock.Bedrock(region="us-east-1")

In [ ]:
number_of_images:int = 10

from batch import Batch

batch = Batch(
    model_id = "amazon.titan-image-generator-v1",
    functions = functions,
    bedrock = _bedrock.client,
)
input_key, output_dir = batch.create_inputs_by(
    prompts = (f"{i+2} dogs running at a park." for i in range(number_of_images))
)
! aws s3 cp $input_key -

ここで、

- S3に入力ファイルがアップロードされたか
- アップロードされたファイルの中身が正しいか

を確認するため、S3内のファイルの中身を覗く

In [ ]:
batch.create_job(input_key, output_dir)

In [ ]:
_bedrock.group_jobs_by_status()

In [ ]:
print(f"ID: {batch.id}")
print(f"Name: {batch.name}")

import utils
utils.wait_until_complete(
    get_status = batch.get_status,
    stopped_status = ('Completed', 'Failed', 'Stopped'),
)

print(f"Error: {batch.error}")
print(f"Total time: {batch.progress_time}")

! aws s3 cp $output_dir$batch.id/manifest.json.out -

In [ ]:
from time import sleep
while True:
    batch.create_job(input_key, output_dir)
    print(_bedrock.group_jobs_by_status())
    sleep(1)

In [ ]:
_bedrock.group_jobs_by_status()

In [ ]:
number_of_images:int = 10

In [ ]:
titan = Batch(
    model_id = "amazon.titan-image-generator-v1",
    functions = functions,
    bedrock = _bedrock.client,
)
input_key, output_dir = batch.create_inputs_by(
    prompts = (f"{i+2} dogs running at a park." for i in range(number_of_images))
)
! aws s3 cp $input_key -

In [ ]:
sd = Batch(
    model_id = "stability.stable-diffusion-xl-v1",
    functions = functions,
    bedrock = _bedrock.client,
)
input_key, output_dir = batch.create_inputs_by(
    prompts = (f"{i+2} dogs running at a park." for i in range(number_of_images))
)
! aws s3 cp $input_key -

ジョブの状態はこちら: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-list.html

ジョブは並列実行されない。

複数のジョブを投げることは可能だが、ジョブが順序保証されて（Undocumented）キューイングされて順次処理される。したがって前のジョブが完了するまで次のジョブは実行されない。

よってジョブの並列実行による高速化はされない。

またバッチ推論にすることでオンラインより高速化もされない。

バッチ推論ジョブの速度はオンデマンド推論を同期処理しているような速度のため、非同期でオンデマンド処理させた方が高速化は可能。

### 画像表示
それぞれのモデルで出力方法が全く異なるので注意

- Titan：単一のJSONLにbase64で全画像が入力情報とセットで埋まっている
- SD：ファイル名に連番が入った一枚一枚のPNG

入力と出力の突き合わせ

- Titan：1行ずつの入力と出力がすぐ対応できるので、突き合わせしやすい
- SD：連番で突き合わせ作業するしかない

In [ ]:
import base64
from io import BytesIO
from PIL import Image

output_dir:str = f"{dir}/{job_id}"

def get_content_binary(key:str):
    output_obj = bucket.Object(key=output_key).get()
    binary_contents = output_obj.get("Body").read()
    return binary_contents


def display_image(image_bytes):
    image = Image.open(BytesIO(image_bytes))
    image.show()

In [ ]:
import json

if "titan" in model_id:
    output_key = f"{output_dir}/{model_id}.jsonl.out"
    binary_contents = get_content_binary(output_key)

    for line in BytesIO(binary_contents):
        content = json.loads(line.decode("utf-8"))
        # finish_reason = content.get("error")
        # if finish_reason is not None: print(f"Image generation error. Error is {finish_reason}")
        print(content.get("modelInput").get("textToImageParams").get("text"))
        base64_image = content.get("modelOutput").get("images")[0]
        base64_bytes = base64_image.encode('ascii')
        image_bytes = base64.b64decode(base64_bytes)
        display_image(image_bytes)
elif "stable" in model_id:
    for record_id in range(number_of_images):
        output_key = f"{output_dir}/{model_id}.{str(record_id).zfill(12)}.1.png"
        binary_contents = get_content_binary(output_key)
        print(output_key)
        display_image(binary_contents)

In [ ]:
# jobs = bedrock.list_model_invocation_jobs().get("invocationJobSummaries")
# from pprint import pprint
# # pprint(jobs)
# for job in jobs:
#     # a = f"{job.get("jobArn")}"
#     print(job)

## 結論

### 入出力データ
- インターフェイスはS3のみ。
- バッチ推論は入出力ファイルサイズ制限がある。モデル間での差分はない。
- https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html#quotas-batch
- 最小も最大も定義されている。

#### 入力データフォーマット
- 一枚一枚に対してはオンデマンド推論と同じ。それをJSONLにしてS3に置く。
- リクエスト枚数が少ないと、ジョブ実行時の序盤にエラーで跳ね返される。
    - Titan: 4枚
    - SD: 10枚

#### 出力データフォーマット
それぞれのモデルで出力方法が全く異なるので注意

- Titan：単一のJSONLにバイナリで全画像が入力情報とセットで埋まっている。大量画像入っているのでデカJSONL
- SD：ファイル名に連番が入った一枚一枚のPNG

入力と出力の突き合わせ

- Titan：1行ずつの入力と出力がすぐ対応できるので、突き合わせしやすい
- SD：連番で突き合わせ作業するしかない

### ジョブ
- ジョブは並列実行されない。
- 複数のジョブを投げることは可能だが、順序保証されて（Undocumented）キューイングされて順次処理される。したがって前のジョブが完了するまで次のジョブは実行されない。
- よってジョブの並列実行による高速化はされない。
- またバッチ推論にすることでオンラインより高速化もされない。
- バッチ推論ジョブの速度はオンデマンド推論を同期処理しているような速度のため、非同期でオンデマンド処理させた方が高速化は可能。

### 結果
600枚全て成功

- Titan: 12 sec / image
- SD: 7 sec /image　← 70分くらい

条件全て揃えられているわけではないので注意